In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.features import preprocessing

In [26]:
# Bypass issues with invoking notebook with server arguments
sys.argv.clear()
sys.argv.append("")

In [18]:
%run utils/attention_graph.py
%run utils/mlflow_query.py
%run utils/loading.py
%run utils/comparison.py
%run utils/ranks.py

In [19]:
mlflow_helper = MlflowHelper(pkl_file=Path("mlflow_run_df.pkl"))
#mlflow_helper.query_runs(pkl_file=Path("mlflow_run_df.pkl"))

Initialized with 86 MLFlow runs from pkl


In [20]:
sequences = pd.read_pickle("../data/sequences_df.pkl")
sequences

,num_logs,num_events,all_events,attributes,fine_log_cluster_template,coarse_log_cluster_template
0,169230,169230,"[[http_status#200.0, , fine_log_cluster_templa...","[[http_status#200.0, , , Hostname#wally113, pr...",[[fine_log_cluster_template#25 nov 2019 15 * *...,[[coarse_log_cluster_template#25 nov 2019 * * ...


In [21]:
huawei_config = preprocessing.HuaweiPreprocessorConfig()
huawei_config.aggregated_log_file = Path("../data/logs_aggregated_concurrent.csv")
huawei_config.traces_root_directory = Path("../data/concurrent_data/traces/")
huawei_config.final_log_file = Path("../data/huawei.pkl")


sequence_preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
huawei_full_data = sequence_preprocessor.load_full_data()

In [22]:
huawei_full_data.columns

Index(['Hostname', 'log_level', 'programname', 'python_module', 'http_status',
       'http_method', '@timestamp', 'Payload', 'http_url',
       'fine_log_cluster_template', 'fine_log_cluster_path',
       'coarse_log_cluster_template', 'coarse_log_cluster_path', 'timestamp',
       'parent_trace_id', 'url_cluster_template', 'url_cluster_path'],
      dtype='object')

# Manually step through program flow

In [28]:
sequences_df = sequence_preprocessor.load_data()

/home/i40/pacev/Domain-Guided-Monitoring/src/features/preprocessing/huawei.py:249: DtypeWarning: Columns (23,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(self.config.aggregated_log_file)
Generating DRAIN clusters from log_df: 100%|██████████| 970/970 [00:00<00:00, 20186.83it/s]
/home/i40/pacev/Domain-Guided-Monitoring/src/features/preprocessing/huawei.py:170: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  merged_df["all_events"] = merged_df[self.relevant_columns].values.tolist()


In [27]:
# TF settings

import tensorflow as tf
import random

tensorflow_seed = 7796
random_seed = 82379498237

tf.random.set_seed(tensorflow_seed)
random.seed(random_seed)



# Collect sequence metadata

In [30]:
from src.features import sequences
from src.features.sequences import transformer

sequence_column_name = sequence_preprocessor.sequence_column_name

transformer_config = sequences.SequenceConfig()
transformer_config.x_sequence_column_name = "fine_log_cluster_template"
transformer_config.y_sequence_column_name = "attributes"
transformer_config.predict_full_y_sequence_wide = True

transformer = transformer.NextPartialSequenceTransformerFromDataframe(transformer_config)

metadata = transformer.collect_metadata(sequences_df, sequence_column_name)